In [1]:
# Import libraries
import os
import random

import torch
from torch.utils.data import Dataset
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import AutoImageProcessor, TimesformerForVideoClassification

import numpy as np
import matplotlib.pyplot as plt

import cv2
import json
from tqdm import tqdm

In [2]:
def identify_device():
    # look for cuda
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    # look for MPS (Apple Silicon GPU)
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        device = torch.device("mps")
        print(f"Using 'Apple Silicion' GPU.")
    else:
        device = torch.device("cpu")
        print(f"No GPU availabe. Using CPU")
    return device

device = identify_device()

Using 'Apple Silicion' GPU.


In [3]:
# Define the dataset class for loading videos and applying transformations
class VideoDataset(Dataset):
    def __init__(self, annotation_file, root_dir, processor, num_frames=64, subset="train"):
        self.annotation_file = annotation_file
        self.root_dir = root_dir
        self.processor = processor
        self.num_frames = num_frames
        self.subset = subset

        # Load annotations
        with open(self.annotation_file, "r") as f:
            self.annotations = json.load(f)

        # Filter videos by subset
        self.video_list = [
            vid for vid, data in self.annotations.items() if data["subset"] == self.subset
        ]
        self.labels = {vid: data["action"][0] for vid, data in self.annotations.items()}

    def __len__(self):
        return len(self.video_list)

    def __getitem__(self, idx):
        video_id = self.video_list[idx]
        label = self.labels[video_id]
        video_path = os.path.join(self.root_dir, f"{video_id}.mp4")

        # Load video frames
        frames = self.load_video_frames(video_path)

        # Apply transformations
        frames = self.apply_transforms(frames)

        # Process frames with processor
        inputs = self.processor(frames, return_tensors="pt")["pixel_values"]

        return inputs.squeeze(0), torch.tensor(label)

    def load_video_frames(self, video_path):
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
          print(f"Error: Unable to open video file {video_path}")
          return []
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frames = []

        # Randomly select starting frame
        start_frame = random.randint(0, max(0, total_frames - self.num_frames))
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        for _ in range(self.num_frames):
            ret, frame = cap.read()
            if not ret:
                break

            # Resize logic
            h, w, c = frame.shape
            if min(w, h) < 226:
                scale = 226 / min(w, h)
                frame = cv2.resize(frame, dsize=(0, 0), fx=scale, fy=scale)
            if max(w, h) > 256:
                scale = 256 / max(w, h)
                frame = cv2.resize(frame, dsize=(0, 0), fx=scale, fy=scale)

            # Convert BGR to RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Append to frames list
            frames.append(frame)

        cap.release()

        # Pad frames if fewer than num_frames
        if len(frames) < self.num_frames:
            pad_frame = frames[0] if random.random() < 0.5 else frames[-1]
            if pad_frame is frames[0]:  # Prepend frames
                while len(frames) < self.num_frames:
                    frames.insert(0, pad_frame)
            else:  # Append frames
                while len(frames) < self.num_frames:
                    frames.append(pad_frame)

        return frames

    def apply_transforms(self, frames):
        """
        Apply consistent transforms to all frames in a video.
        """
        if self.subset == "train" or self.subset == "val":
            # Generate a single random crop and flip decision for the entire video
            crop_transform = transforms.RandomCrop(224)
            flip_transform = transforms.RandomHorizontalFlip(p=0.5)
            random_flip = random.random() < 0.5  # Determine if flipping is applied

            # Define a manual crop region using RandomCrop's get_params
            first_frame = transforms.ToPILImage()(frames[0])
            i, j, h, w = transforms.RandomCrop.get_params(first_frame, output_size=(224, 224))

            def consistent_transform(frame):
                frame = transforms.ToPILImage()(frame)
                frame = transforms.functional.crop(frame, i, j, h, w)  # Apply the same crop
                if random_flip:
                    frame = transforms.functional.hflip(frame)  # Apply the same flip
                return transforms.ToTensor()(frame)

            # Apply the consistent transforms to all frames
            return [consistent_transform(frame) for frame in frames]
        else:  # For testing
            pil_transforms = transforms.Compose([
                transforms.ToPILImage(),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
            ])
            return [pil_transforms(frame) for frame in frames]

Train Last 3 Layers with 64 frames

In [ ]:
# Configuration
annotation_file = 'nslt_100.json'
root_dir = 'WLASL100_videos'
pretrained_weights = "facebook/timesformer-base-finetuned-k400"
processor = AutoImageProcessor.from_pretrained(pretrained_weights, do_rescale=False)
model = TimesformerForVideoClassification.from_pretrained(pretrained_weights, num_labels=100, ignore_mismatched_sizes=True)

model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),  # Apply 50% dropout
    nn.Linear(model.classifier.in_features, model.classifier.out_features)
)

# Training Dataset (random cropping and flipping)
train_dataset = VideoDataset(
    annotation_file=annotation_file,
    root_dir=root_dir,
    processor=processor,
    num_frames=64,
    subset="train"
)

# Validation Dataset (random cropping and flipping)
val_dataset = VideoDataset(
    annotation_file=annotation_file,
    root_dir=root_dir,
    processor=processor,
    num_frames=64,
    subset="val"
)

# Testing Dataset (center cropping)
test_dataset = VideoDataset(
    annotation_file=annotation_file,
    root_dir=root_dir,
    processor=processor,
    num_frames=64,
    subset="test"
)

# Combine train and val datasets for training as Li et al. (2020)
combined_train_dataset = torch.utils.data.ConcatDataset([train_dataset, val_dataset])

# DataLoaders
train_loader = DataLoader(combined_train_dataset, batch_size=4, shuffle=True, num_workers=12)
val_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=12)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last three layers
for name, param in model.named_parameters():
    if "encoder.layer.10" in name or "encoder.layer.11" in name or "classifier" in name:
        param.requires_grad = True

# Move model to device
model.to(device)

print("Trainable Parameters:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", patience=1, factor=0.3)

best_val_accuracy = 0
epochs_no_improve = 0
epochs = 20

for epoch in range(epochs):
    model.train()
    total_loss = 0

    print(f"Epoch {epoch + 1}/{epochs}")

    # Use tqdm to create a progress bar for the training loop
    with tqdm(total=len(train_loader), desc="Training", unit="batch") as pbar:
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(pixel_values=inputs, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            # Update the progress bar
            pbar.update(1)
            pbar.set_postfix({"Loss": f"{loss.item():.4f}"})

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")
    print(f"Current learning rate: {optimizer.param_groups[0]['lr']}")

    # Validation phase
    model.eval()
    total_correct = 0
    total_samples = 0
    with tqdm(total=len(val_loader), desc="Validation", unit="batch") as pbar:
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(pixel_values=inputs)
                predictions = outputs.logits.argmax(dim=-1)
                total_correct += (predictions == labels).sum().item()
                total_samples += labels.size(0)
                pbar.update(1)

    val_accuracy = total_correct / total_samples
    print(f"Validation Accuracy: {val_accuracy:.4f}")

    scheduler.step(val_accuracy)

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_no_improve = 0
        # Save the best model
        torch.save(model.state_dict(), "best_model_TSF_3_layers_64_frames.pt")
        print("Validation accuracy improved, saving model.")
    else:
        epochs_no_improve += 1
        print(f"No improvement for {epochs_no_improve} epochs.")

## Testing

In [4]:
# Configuration
annotation_file = 'nslt_100.json'
root_dir = 'WLASL100_videos'
pretrained_weights = "facebook/timesformer-base-finetuned-k400"
processor = AutoImageProcessor.from_pretrained(pretrained_weights, do_rescale=False)
model = TimesformerForVideoClassification.from_pretrained(pretrained_weights, num_labels=100, ignore_mismatched_sizes=True)

model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),  # Apply 50% dropout
    nn.Linear(model.classifier.in_features, model.classifier.out_features)
)

# Training Dataset (random cropping and flipping)
train_dataset = VideoDataset(
    annotation_file=annotation_file,
    root_dir=root_dir,
    processor=processor,
    num_frames=64,
    subset="train"
)

# Validation Dataset (random cropping and flipping)
val_dataset = VideoDataset(
    annotation_file=annotation_file,
    root_dir=root_dir,
    processor=processor,
    num_frames=64,
    subset="val"
)

# Testing Dataset (center cropping)
test_dataset = VideoDataset(
    annotation_file=annotation_file,
    root_dir=root_dir,
    processor=processor,
    num_frames=64,
    subset="test"
)

# Combine train and val datasets for training as Li et al. (2020)
combined_train_dataset = torch.utils.data.ConcatDataset([train_dataset, val_dataset])

# DataLoaders
train_loader = DataLoader(combined_train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

Some weights of TimesformerForVideoClassification were not initialized from the model checkpoint at facebook/timesformer-base-finetuned-k400 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([400, 768]) in the checkpoint and torch.Size([100, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([400]) in the checkpoint and torch.Size([100]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.load_state_dict(torch.load("best_model_TSF_3_layers_64_frames.pt", map_location=device, weights_only=False)) # Load respective model
model.to(device)
model.eval()

correct_top1 = 0
correct_top5 = 0
correct_top10 = 0
total = 0

with torch.no_grad():
    for videos, labels in val_loader:
        videos = videos.squeeze(1).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(pixel_values=videos)
        logits = outputs.logits
        _, top1_preds = logits.topk(1, dim=-1)  # Top-1 predictions
        _, top5_preds = logits.topk(5, dim=-1)  # Top-5 predictions
        _, top10_preds = logits.topk(10, dim=-1)  # Top-10 predictions

        # Update metrics
        correct_top1 += (top1_preds.squeeze() == labels).sum().item()
        correct_top5 += sum([labels[i] in top5_preds[i] for i in range(len(labels))])
        correct_top10 += sum([labels[i] in top10_preds[i] for i in range(len(labels))])
        total += labels.size(0)

# Calculate accuracies
top1_accuracy = correct_top1 / total
top5_accuracy = correct_top5 / total
top10_accuracy = correct_top10 / total

print(f"Top-1 Accuracy: {top1_accuracy:.4f}")
print(f"Top-5 Accuracy: {top5_accuracy:.4f}")
print(f"Top-10 Accuracy: {top10_accuracy:.4f}")

## 16 Frame evenly and all layers

In [3]:
# Update dataset class to sample frames evenly across the video
class VideoDataset(Dataset):
    def __init__(self, annotation_file, root_dir, processor, num_frames=64, subset="train"):
        self.annotation_file = annotation_file
        self.root_dir = root_dir
        self.processor = processor
        self.num_frames = num_frames
        self.subset = subset

        # Load annotations
        with open(self.annotation_file, "r") as f:
            self.annotations = json.load(f)

        # Filter videos by subset
        self.video_list = [
            vid for vid, data in self.annotations.items() if data["subset"] == self.subset
        ]
        self.labels = {vid: data["action"][0] for vid, data in self.annotations.items()}

    def __len__(self):
        return len(self.video_list)

    def __getitem__(self, idx):
        video_id = self.video_list[idx]
        label = self.labels[video_id]
        video_path = os.path.join(self.root_dir, f"{video_id}.mp4")

        # Load video frames
        frames = self.load_video_frames(video_path)

        # Apply transformations
        frames = self.apply_transforms(frames)

        # Process frames with processor
        inputs = self.processor(frames, return_tensors="pt")["pixel_values"]

        return inputs.squeeze(0), torch.tensor(label)

    def load_video_frames(self, video_path):
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Cannot open video file {video_path}.")
            return []

        frames = []
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Calculate step size to evenly distribute frames
        if total_frames > 1:
            step = max(1, total_frames // self.num_frames)
        else:
            step = 1

        for i in range(0, step * self.num_frames, step):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if not ret:
                break
            h, w, _ = frame.shape
            if min(w, h) < 226:
                scale = 226 / min(w, h)
                frame = cv2.resize(frame, None, fx=scale, fy=scale)
            if max(w, h) > 256:
                scale = 256 / max(w, h)
                frame = cv2.resize(frame, None, fx=scale, fy=scale)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)

        cap.release()

        # Handle cases with fewer frames than expected
        if len(frames) < self.num_frames:
            pad_choice = random.random()
            if pad_choice < 0.5 and frames:
                pad_frame = frames[0]
                while len(frames) < self.num_frames:
                    frames.insert(0, pad_frame)
            elif frames:
                pad_frame = frames[-1]
                while len(frames) < self.num_frames:
                    frames.append(pad_frame)

        # In case frames were empty
        while len(frames) < self.num_frames:
            frames.append(np.zeros((224, 224, 3), dtype=np.uint8))

        return frames

    def apply_transforms(self, frames):
        """
        Apply consistent transforms to all frames in a video.
        """
        if self.subset == "train" or self.subset == "val":
            # Generate a single random crop and flip decision for the entire video
            crop_transform = transforms.RandomCrop(224)
            flip_transform = transforms.RandomHorizontalFlip(p=0.5)
            random_flip = random.random() < 0.5  # Determine if flipping is applied

            # Define a manual crop region using RandomCrop's get_params
            first_frame = transforms.ToPILImage()(frames[0])
            i, j, h, w = transforms.RandomCrop.get_params(first_frame, output_size=(224, 224))

            def consistent_transform(frame):
                frame = transforms.ToPILImage()(frame)
                frame = transforms.functional.crop(frame, i, j, h, w)  # Apply the same crop
                if random_flip:
                    frame = transforms.functional.hflip(frame)  # Apply the same flip
                return transforms.ToTensor()(frame)

            # Apply the consistent transforms to all frames
            return [consistent_transform(frame) for frame in frames]
        else:  # For testing
            pil_transforms = transforms.Compose([
                transforms.ToPILImage(),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
            ])
            return [pil_transforms(frame) for frame in frames]

In [ ]:
# Configuration
annotation_file = 'nslt_100.json'
root_dir = 'WLASL100_videos'
pretrained_weights = "facebook/timesformer-base-finetuned-k400"
processor = AutoImageProcessor.from_pretrained(pretrained_weights, do_rescale=False)
model = TimesformerForVideoClassification.from_pretrained(pretrained_weights, num_labels=100, ignore_mismatched_sizes=True)

model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),  # Apply 50% dropout
    nn.Linear(model.classifier.in_features, model.classifier.out_features)
)

# Training Dataset (random cropping and flipping)
train_dataset = VideoDataset(
    annotation_file=annotation_file,
    root_dir=root_dir,
    processor=processor,
    num_frames=16,
    subset="train"
)

# Validation Dataset (random cropping and flipping)
val_dataset = VideoDataset(
    annotation_file=annotation_file,
    root_dir=root_dir,
    processor=processor,
    num_frames=16,
    subset="val"
)

# Testing Dataset (center cropping)
test_dataset = VideoDataset(
    annotation_file=annotation_file,
    root_dir=root_dir,
    processor=processor,
    num_frames=16,
    subset="test"
)

# Combine train and val datasets for training as Li et al. (2020)
combined_train_dataset = torch.utils.data.ConcatDataset([train_dataset, val_dataset])

# DataLoaders
train_loader = DataLoader(combined_train_dataset, batch_size=4, shuffle=True, num_workers=12)
val_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=12)

# Make all layers trainable
for param in model.parameters():
    param.requires_grad = True

# Move model to device
model.to(device)

print("Trainable Parameters:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

optimizer = optim.AdamW(model.parameters(), lr=0.00001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", patience=1, factor=0.3)

best_val_accuracy = 0
epochs_no_improve = 0
epochs = 20

for epoch in range(epochs):
    model.train()
    total_loss = 0

    print(f"Epoch {epoch + 1}/{epochs}")

    # Use tqdm to create a progress bar for the training loop
    with tqdm(total=len(train_loader), desc="Training", unit="batch") as pbar:
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(pixel_values=inputs, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            # Update the progress bar
            pbar.update(1)
            pbar.set_postfix({"Loss": f"{loss.item():.4f}"})

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")
    print(f"Current learning rate: {optimizer.param_groups[0]['lr']}")

    # Validation phase
    model.eval()
    total_correct = 0
    total_samples = 0
    with tqdm(total=len(val_loader), desc="Validation", unit="batch") as pbar:
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(pixel_values=inputs)
                predictions = outputs.logits.argmax(dim=-1)
                total_correct += (predictions == labels).sum().item()
                total_samples += labels.size(0)
                pbar.update(1)

    val_accuracy = total_correct / total_samples
    print(f"Validation Accuracy: {val_accuracy:.4f}")

    scheduler.step(val_accuracy)

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_no_improve = 0
        # Save the best model
        torch.save(model.state_dict(), "best_model_TSF_all_layers_16_frames.pt")
        print("Validation accuracy improved, saving model.")
    else:
        epochs_no_improve += 1
        print(f"No improvement for {epochs_no_improve} epochs.")

In [ ]:
model.load_state_dict(torch.load("best_model_TSF_all_layers_16_frames.pt", map_location=device, weights_only=False)) # Load respective model
model.to(device)
model.eval()

correct_top1 = 0
correct_top5 = 0
correct_top10 = 0
total = 0

with torch.no_grad():
    for videos, labels in val_loader:
        videos = videos.squeeze(1).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(pixel_values=videos)
        logits = outputs.logits
        _, top1_preds = logits.topk(1, dim=-1)  # Top-1 predictions
        _, top5_preds = logits.topk(5, dim=-1)  # Top-5 predictions
        _, top10_preds = logits.topk(10, dim=-1)  # Top-10 predictions

        # Update metrics
        correct_top1 += (top1_preds.squeeze() == labels).sum().item()
        correct_top5 += sum([labels[i] in top5_preds[i] for i in range(len(labels))])
        correct_top10 += sum([labels[i] in top10_preds[i] for i in range(len(labels))])
        total += labels.size(0)

# Calculate accuracies
top1_accuracy = correct_top1 / total
top5_accuracy = correct_top5 / total
top10_accuracy = correct_top10 / total

print(f"Top-1 Accuracy: {top1_accuracy:.4f}")
print(f"Top-5 Accuracy: {top5_accuracy:.4f}")
print(f"Top-10 Accuracy: {top10_accuracy:.4f}")

## 16 frames evenly and last three layers

In [ ]:
# Configuration
annotation_file = 'nslt_100.json'
root_dir = 'WLASL100_videos'
pretrained_weights = "facebook/timesformer-base-finetuned-k400"
processor = AutoImageProcessor.from_pretrained(pretrained_weights, do_rescale=False)
model = TimesformerForVideoClassification.from_pretrained(pretrained_weights, num_labels=100, ignore_mismatched_sizes=True)

model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),  # Apply 50% dropout
    nn.Linear(model.classifier.in_features, model.classifier.out_features)
)

# Training Dataset (random cropping and flipping)
train_dataset = VideoDataset(
    annotation_file=annotation_file,
    root_dir=root_dir,
    processor=processor,
    num_frames=16,
    subset="train"
)

# Validation Dataset (random cropping and flipping)
val_dataset = VideoDataset(
    annotation_file=annotation_file,
    root_dir=root_dir,
    processor=processor,
    num_frames=16,
    subset="val"
)

# Testing Dataset (center cropping)
test_dataset = VideoDataset(
    annotation_file=annotation_file,
    root_dir=root_dir,
    processor=processor,
    num_frames=16,
    subset="test"
)

# Combine train and val datasets for training as Li et al. (2020)
combined_train_dataset = torch.utils.data.ConcatDataset([train_dataset, val_dataset])

# DataLoaders
train_loader = DataLoader(combined_train_dataset, batch_size=4, shuffle=True, num_workers=12)
val_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=12)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last three layers
for name, param in model.named_parameters():
    if "encoder.layer.10" in name or "encoder.layer.11" in name or "classifier" in name:
        param.requires_grad = True

# Move model to device
model.to(device)

print("Trainable Parameters:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

optimizer = optim.AdamW(model.parameters(), lr=0.00001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", patience=1, factor=0.3)

best_val_accuracy = 0
epochs_no_improve = 0
epochs = 20

for epoch in range(epochs):
    model.train()
    total_loss = 0

    print(f"Epoch {epoch + 1}/{epochs}")

    # Use tqdm to create a progress bar for the training loop
    with tqdm(total=len(train_loader), desc="Training", unit="batch") as pbar:
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(pixel_values=inputs, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            # Update the progress bar
            pbar.update(1)
            pbar.set_postfix({"Loss": f"{loss.item():.4f}"})

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")
    print(f"Current learning rate: {optimizer.param_groups[0]['lr']}")

    # Validation phase
    model.eval()
    total_correct = 0
    total_samples = 0
    with tqdm(total=len(val_loader), desc="Validation", unit="batch") as pbar:
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(pixel_values=inputs)
                predictions = outputs.logits.argmax(dim=-1)
                total_correct += (predictions == labels).sum().item()
                total_samples += labels.size(0)
                pbar.update(1)

    val_accuracy = total_correct / total_samples
    print(f"Validation Accuracy: {val_accuracy:.4f}")

    scheduler.step(val_accuracy)

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_no_improve = 0
        # Save the best model
        torch.save(model.state_dict(), "best_model_TSF_3_layers_16_frames.pt")
        print("Validation accuracy improved, saving model.")
    else:
        epochs_no_improve += 1
        print(f"No improvement for {epochs_no_improve} epochs.")

In [ ]:
model.load_state_dict(torch.load("best_model_TSF_3_layers_16_frames", map_location=device, weights_only=False)) # Load respective model
model.to(device)
model.eval()

correct_top1 = 0
correct_top5 = 0
correct_top10 = 0
total = 0

with torch.no_grad():
    for videos, labels in val_loader:
        videos = videos.squeeze(1).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(pixel_values=videos)
        logits = outputs.logits
        _, top1_preds = logits.topk(1, dim=-1)  # Top-1 predictions
        _, top5_preds = logits.topk(5, dim=-1)  # Top-5 predictions
        _, top10_preds = logits.topk(10, dim=-1)  # Top-10 predictions

        # Update metrics
        correct_top1 += (top1_preds.squeeze() == labels).sum().item()
        correct_top5 += sum([labels[i] in top5_preds[i] for i in range(len(labels))])
        correct_top10 += sum([labels[i] in top10_preds[i] for i in range(len(labels))])
        total += labels.size(0)

# Calculate accuracies
top1_accuracy = correct_top1 / total
top5_accuracy = correct_top5 / total
top10_accuracy = correct_top10 / total

print(f"Top-1 Accuracy: {top1_accuracy:.4f}")
print(f"Top-5 Accuracy: {top5_accuracy:.4f}")
print(f"Top-10 Accuracy: {top10_accuracy:.4f}")